# 2021/10/21
The jupypter notebook is used to test the pipleine for the ENN + SMOTE + random forest pipeline intergration from sklearn for single parameter


In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import pandas as pd
import numpy as np
import sys, os

# Load the data

In [4]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
#df=pd.read_csv(os.path.join(input_path, 'total.Megalodon.per_read.prob.bed.gz'),compression='gzip', sep='\t')
df=pd.read_csv(os.path.join(input_path, 'total.test.bed'), sep='\t')
print("Data is loading!")

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

df

Data is loading!


,chr,start,read_id,strand,5hmC_prob,5mC_prob,5C_prob,label
0,KJ477685.1,204,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.435397,0.085058,0.479545,2
1,KJ477685.1,261,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.342189,0.500821,0.156990,2
2,KJ477685.1,261,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.577034,0.057676,0.365290,2
3,KJ477685.1,266,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.065316,0.861569,0.073115,2
4,KJ477685.1,266,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.794235,0.028885,0.176880,2
...,...,...,...,...,...,...,...,...
2492,J02459.1,60,bb826a6c-c88b-45ac-9984-8b9d6bfb2f11,-,0.054875,0.013023,0.932103,0
2493,J02459.1,60,bb91267a-6661-4248-a084-554f231398c1,-,0.030720,0.004392,0.964888,0
2494,J02459.1,60,bb9cee29-89d0-4a68-a4b4-8fe5c000289f,-,0.032703,0.006934,0.960363,0
2495,J02459.1,60,bbee6b4a-acac-4db9-bbb8-2f379918b146,-,0.035734,0.003182,0.961085,0


In [5]:
df['label'].value_counts()

1    999
0    999
2    499
Name: label, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, GridSearchCV
#from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [12]:
#Load the feature
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test'
#df=pd.read_csv(os.path.join(input_path, 'total.Megalodon.per_read.prob.bed.gz'),compression='gzip', sep='\t')
df=pd.read_csv(os.path.join(input_path, 'total.test.bed'), sep='\t')

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

X = df_feature
y = df_class

#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [13]:
from collections import Counter
print("Before the pipeline:\n y_train:{},\n y_test: {}".format(Counter(y_train), Counter(y_test)))

Before the pipeline:
 y_train:Counter({0: 799, 1: 799, 2: 399}),
 y_test: Counter({0: 200, 1: 200, 2: 100})


## Resampling strategy

In [17]:
## Define resampling strategy
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import EditedNearestNeighbours, RandomUnderSampler

### RandomUnderSampler

In [21]:
sampler = RandomUnderSampler(random_state = 42, sampling_strategy={0: 500, 1:500, 2:300})

X_res, y_res = sampler.fit_resample(X_train, y_train)

print(Counter(y_res))

Counter({0: 500, 1: 500, 2: 300})


### ENN

In [28]:
from imblearn.under_sampling import EditedNearestNeighbours
trans = EditedNearestNeighbours()

n_neighbors = [20, 50, 100, 1000, 2000, 5000]
for n in n_neighbors:
    print(n)
    ENN = EditedNearestNeighbours(n_neighbors = n, n_jobs = -1)
    X_resampled, y_resampled = ENN.fit_resample(X_train, y_train)
    print(Counter(y_resampled))

20
Counter({1: 660, 0: 525, 2: 399})
50
Counter({1: 565, 2: 399, 0: 385})
100
Counter({1: 450, 2: 399, 0: 236})
1000
Counter({2: 399})
2000


ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1997, n_neighbors = 2001

In [ ]:
'EditedNearestNeighbours' object has no attribute 'fit_sample'

In [19]:
# Make the resampling strategy: RandomUnderSampler -> SMOTE
sampler = make_pipeline(
    RandomUnderSampler(random_state = 42, sampling_strategy={0: 500, 1:500, 2:270}),
    SMOTE(sampling_strategy='not majority',
          k_neighbors=100,
          random_state = 42,
          n_jobs = -1))

X_res, y_res = sampler.fit_resample(X_train, y_train)

print(Counter(y_res))

Counter({0: 500, 1: 500, 2: 500})


In [20]:
# Make the resampling strategy: ENN -> SMOTE
ENN_SMOTE_sampler = make_pipeline(
    EditedNearestNeighbours(n_neighbors = 100,
                            n_jobs = -1),
    SMOTE(sampling_strategy='not majority',
          k_neighbors=100,
          random_state = 42,
          n_jobs = -1))

X_res_train, y_res_train = ENN_SMOTE_sampler.fit_resample(X_train, y_train)

print(Counter(y_res_train))

Counter({0: 450, 1: 450, 2: 450})


## Test with single parameter

I used the best parameter from 2021/10/13 for a test

{'rfr__bootstrap': False, 'rfr__max_depth': 25, 'rfr__max_features': 'auto', 'rfr__min_samples_leaf': 2, 'rfr__min_samples_split': 2, 'rfr__n_estimators': 120}  

In [24]:
from imblearn.pipeline import Pipeline as imbpipeline
##define random forest model
rf_model = RandomForestClassifier(
    n_estimators=10,
    max_depth=25,
    max_features = 'auto',
    min_samples_split=2,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1)

under = RandomUnderSampler(random_state = 42, sampling_strategy={0: 500, 1:500, 2:200})

over = SMOTE(sampling_strategy='not majority',
             k_neighbors=100,
             random_state = 42,
             n_jobs = -1)

steps = [('u', under), ('o', over), ('m', rf_model)]

pipeline = imbpipeline(steps=steps)

In [25]:
rf_result = pipeline.fit(X_train, y_train)
y_true, y_pred = y_test, pipeline.predict(X_test)

In [28]:
from sklearn.model_selection import cross_val_score
from numpy import mean
scores = cross_val_score(pipeline, X_train, y_train, scoring='f1_macro', cv=stratified_kfold, n_jobs=-1)
score = mean(scores)
print('F1 Score: %.3f' % score)

F1 Score: 0.902


## Random+SMOTE: Test with multiple parameter

In [27]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV

from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Define cross-validation fold
stratified_kfold = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

##define random forest model
#https://stackoverflow.com/a/51493479
###Build the pipeline
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
under = RandomUnderSampler(random_state = 42)

over = SMOTE(sampling_strategy='not majority', random_state = 42, n_jobs = -1)

steps = [('u', under), ('o', over), ('m', rf_model)]

pipeline = imbpipeline(steps=steps)

In [28]:
## Get the parameter dict keys
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'u', 'o', 'm', 'u__random_state', 'u__replacement', 'u__sampling_strategy', 'o__k_neighbors', 'o__n_jobs', 'o__random_state', 'o__sampling_strategy', 'm__bootstrap', 'm__ccp_alpha', 'm__class_weight', 'm__criterion', 'm__max_depth', 'm__max_features', 'm__max_leaf_nodes', 'm__max_samples', 'm__min_impurity_decrease', 'm__min_impurity_split', 'm__min_samples_leaf', 'm__min_samples_split', 'm__min_weight_fraction_leaf', 'm__n_estimators', 'm__n_jobs', 'm__oob_score', 'm__random_state', 'm__verbose', 'm__warm_start'])

In [29]:
#https://stackoverflow.com/questions/51480776/how-to-implement-ratio-based-smote-oversampling-while-cv-ing-dataset
params = {'u__sampling_strategy' : [{0: 500, 1:500, 2:100}, {0: 500, 1:200, 2:100}],
          'o__k_neighbors':[20, 30, 40],
          'm__max_depth' : list(range(2,5)),
          'm__max_features' : ['auto','sqrt'],
          'm__bootstrap' : [True, False]
         }


grid = RandomizedSearchCV(estimator = pipeline, 
                          param_distributions = params, 
                          scoring='f1_macro', 
                          cv=stratified_kfold, 
                          verbose=1,   
                          n_iter=10,
                          return_train_score=True, 
                          n_jobs=-1)

In [30]:
result = grid.fit(X_train, y_train)
y_true, y_pred = y_test, grid.predict(X_test)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


## ENN+SMOTE: Test with multiple parameter

In [101]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV

from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours

# Define cross-validation fold
stratified_kfold = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

##define random forest model
#https://stackoverflow.com/a/51493479
###Build the pipeline
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
under = EditedNearestNeighbours(n_jobs = -1)
over = SMOTE(sampling_strategy='not majority', random_state = 42, n_jobs = -1)
steps = [('u', under), ('o', over), ('m', rf_model)]

pipeline = imbpipeline(steps=steps)
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'u', 'o', 'm', 'u__kind_sel', 'u__n_jobs', 'u__n_neighbors', 'u__sampling_strategy', 'o__k_neighbors', 'o__n_jobs', 'o__random_state', 'o__sampling_strategy', 'm__bootstrap', 'm__ccp_alpha', 'm__class_weight', 'm__criterion', 'm__max_depth', 'm__max_features', 'm__max_leaf_nodes', 'm__max_samples', 'm__min_impurity_decrease', 'm__min_impurity_split', 'm__min_samples_leaf', 'm__min_samples_split', 'm__min_weight_fraction_leaf', 'm__n_estimators', 'm__n_jobs', 'm__oob_score', 'm__random_state', 'm__verbose', 'm__warm_start'])

In [104]:
#https://stackoverflow.com/questions/51480776/how-to-implement-ratio-based-smote-oversampling-while-cv-ing-dataset
params = {'u__n_neighbors' : [20,50,100],
          'o__k_neighbors':[20, 30, 40],
          'm__max_depth' : list(range(2,5)),
          'm__max_features' : ['auto','sqrt'],
          'm__bootstrap' : [True, False]
         }

grid = RandomizedSearchCV(estimator = pipeline, 
                          param_distributions = params, 
                          scoring='f1_macro', 
                          cv=stratified_kfold, 
                          verbose=3,   
                          n_iter=10,
                          return_train_score=True, 
                          n_jobs=-1)

In [105]:
result = grid.fit(X_train, y_train)
y_true, y_pred = y_test, grid.predict(X_test)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END m__bootstrap=True, m__max_depth=4, m__max_features=auto, o__k_neighbors=40, u__n_neighbors=50;, score=(train=0.789, test=0.774) total time=   2.1s
[CV 2/3] END m__bootstrap=True, m__max_depth=4, m__max_features=auto, o__k_neighbors=40, u__n_neighbors=50;, score=(train=0.739, test=0.744) total time=   1.0s
[CV 3/3] END m__bootstrap=True, m__max_depth=4, m__max_features=auto, o__k_neighbors=40, u__n_neighbors=50;, score=(train=0.769, test=0.771) total time=   1.0s
[CV 1/3] END m__bootstrap=True, m__max_depth=2, m__max_features=sqrt, o__k_neighbors=40, u__n_neighbors=100;, score=(train=0.715, test=0.674) total time=   3.9s
[CV 2/3] END m__bootstrap=True, m__max_depth=2, m__max_features=sqrt, o__k_neighbors=40, u__n_neighbors=100;, score=(train=0.714, test=0.719) total time=   2.0s
[CV 3/3] END m__bootstrap=True, m__max_depth=2, m__max_features=sqrt, o__k_neighbors=40, u__n_neighbors=100;, score=(train=0.676, test=0.